Interacting Proteins

In [66]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import sys
sys.path.append('C:\\Users\\brittany henderson\\GitHub\\WhenMutationsDontMatter\\')
import plot_utils as pu

In [2]:
gbm = cptac.Gbm()
endo = cptac.Endometrial()
h = cptac.Hnscc()
l = cptac.Luad()
o = cptac.Ovarian()
col = cptac.Colon()
b = cptac.Brca()

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 1)


cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 4)


In [3]:
gene = 'PTEN'

In [4]:
ip = u.get_interacting_proteins('PTEN')

# RAS-RAF-MEK-ERK MAPK Pathway
ip.append('GRB2')
ip.append('SHC1') # (SHC in paper)
ip.append('SOS1') # (SOS in the paper) *GEF for RAS "recruitment to the plasma membrane requires GRB2, PIP2, PA" 
ip.append('KRAS') # (RAS in paper) possibly add "RAS regulators such as PHLPP, SHP-2, and NF-2"
ip.append('RAF1') # (RAF-1 in paper)
ip.append('MAP2K1') #(MEK1 protein name in paper)
ip.append('MAP2K2 ') # (MEK2 in paper)
ip.append('MAPK3') # (ERK1 in paper) https://www.uniprot.org/uniprot/P27361
ip.append('MAPK1') # (ERK2 in paper)

#mTOR Pathway
ip.append('GAB1')
#ip.append('PI3K')
ip.append('AKT1') # (AKT in paper) *binds PIP3 (see info for isoform AKT1 at: https://www.uniprot.org/uniprot/P31749)
ip.append('PDK1') #activate AKT
ip.append('MTORC2') #activate AKT
ip.append('GSK3B') # (GSK-3B in paper)inhibit cyclin D (akt inhibits GSK3B)
ip.append('FOXO1') #(FOXO in paper)
ip.append('TSC2') # repress mTOR, AKT inhibits
ip.append('MTOR') # promote cyclin D
ip.append('BAD')
ip.append('CAS9')

ip.append('CCND1') # Cyclin D1 -> G1/S 


singles = set(ip)
ip = list(singles)
print(len(ip))
#ip

46


In [5]:
mut_type_gbm = gbm.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = gbm.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ip)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_gbm = mut_type_gbm[['Mutation']] 
merged = ip_df.join(mut_type_gbm) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()
#del_wt

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 72 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-01834 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, MAP2K2 , MTORC2, PIK3R3 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PTEN gene (C:\

Deletion          81
Wildtype_Tumor    16
Name: Mutation, dtype: int64

In [6]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
g_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if g_sig is not None:
    g_sig_list = list(g_sig.Comparison)
else: 
    g_sig_list = None
print('significant pvals: \n',g_sig)

# Get all pvals
g = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
g_pval = g.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Gbm_P_Value'})
#g_pval = g_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)
g_pval.head()

significant pvals: 
           Comparison       P_Value
0    PTEN_proteomics  1.219523e-07
1   MAPK3_proteomics  1.020660e-06
2  PIK3CD_proteomics  1.030815e-03
3    GRB2_proteomics  1.089951e-03
4  PIK3C3_proteomics  1.225949e-03
5    USP7_proteomics  4.017792e-03
6  MAP2K1_proteomics  4.138456e-03
7  PIK3R1_proteomics  4.282296e-03


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corre

,Proteomics,Gbm_P_Value
0,PTEN_proteomics,1.219523e-07
1,MAPK3_proteomics,1.020660e-06
2,PIK3CD_proteomics,1.030815e-03
3,GRB2_proteomics,1.089951e-03
4,PIK3C3_proteomics,1.225949e-03


In [7]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

gbm_d = {}

for prot in ip:
    dif_gbm = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    gbm_d[prot+'_proteomics'] = dif_gbm
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif_gbm)

gbm_df = pd.DataFrame.from_dict(gbm_d, orient='index', columns=['Gbm_Median'])
gbm_df = gbm_df.reset_index().rename(columns={'index':'Proteomics'})
gbm_df.head()

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


,Proteomics,Gbm_Median
0,BAD_proteomics,0.087843
1,USP13_proteomics,-0.111700
2,PIK3R1_proteomics,0.188390
3,PDGFRB_proteomics,-0.028576
4,MAPK1_proteomics,0.206357


In [8]:
g_merged = g_pval.merge(gbm_df, on='Proteomics',how='inner')
g_merged.head()

,Proteomics,Gbm_P_Value,Gbm_Median
0,PTEN_proteomics,1.219523e-07,0.511726
1,MAPK3_proteomics,1.020660e-06,0.665105
2,PIK3CD_proteomics,1.030815e-03,0.492099
3,GRB2_proteomics,1.089951e-03,0.312146
4,PIK3C3_proteomics,1.225949e-03,0.102518


Endo

In [9]:
#Endo ttest

mut_type_gbm = endo.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = endo.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ip)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_gbm = mut_type_gbm[['Mutation']] 
merged = ip_df.join(mut_type_gbm) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Nonsense_Mutation','Frame_Shift_Ins','Frame_Shift_Del']
get = merged['Mutation'].isin(compare)
trunc_wt = merged[get]
trunc_wt['Mutation'] = np.where(
                trunc_wt['Mutation'] == 'Wildtype_Tumor', 'Wildtype_Tumor', 'Truncation')
trunc_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 20 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, MAP2K2 , MDM2, MTORC2, PIK3R3 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 7)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 69 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 7)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value i

Truncation        53
Wildtype_Tumor    15
Name: Mutation, dtype: int64

In [10]:
cols = list(trunc_wt.columns[:-1])

# Get only sig sites
e_sig = u.wrap_ttest(trunc_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if e_sig is not None:
    e_sig_list = list(e_sig.Comparison)
else: 
    e_sig_list = None
print('significant pvals: \n',e_sig)

# Get all pvals
e_pval = u.wrap_ttest(trunc_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
e_pval = e_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'En_P_Value'})
#e_pval

significant pvals: 
            Comparison   P_Value
0     PTEN_proteomics  0.000640
1     TP53_proteomics  0.001246
2   PIK3CA_proteomics  0.001761
3  CSNK2A1_proteomics  0.002364


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corre

In [11]:
t = trunc_wt[trunc_wt.Mutation == "Truncation"]
wt = trunc_wt[trunc_wt.Mutation == "Wildtype_Tumor"]
trunc_med = t.median()
wt_med = wt.median()

en_d = {}

for prot in ip:
    dif = wt_med[prot+'_proteomics'] - trunc_med[prot+'_proteomics']
    en_d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', trunc_med[prot+'_proteomics'], '=',dif)

en_df = pd.DataFrame.from_dict(en_d, orient='index', columns=['En_Median'])
en_df = en_df.reset_index().rename(columns={'index':'Proteomics'})
#en_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [12]:
e_merged = e_pval.merge(en_df, on='Proteomics',how='inner')
e_merged.head()

,Proteomics,En_P_Value,En_Median
0,PTEN_proteomics,0.000640,0.74660
1,TP53_proteomics,0.001246,0.80260
2,PIK3CA_proteomics,0.001761,0.35300
3,CSNK2A1_proteomics,0.002364,0.10412
4,EGFR_proteomics,0.006128,-0.52600


Ovarian

In [13]:
#o = cptac.Ovarian()

In [14]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
gene = 'PTEN'

mut_type_gbm = o.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = o.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ip)
prot_and_mutations = o.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations.loc[:,~prot_and_mutations.columns.duplicated()] # drop duplicated columns
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_gbm = mut_type_gbm[['Mutation']] 
merged = ip_df.join(mut_type_gbm) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 98 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 01OV029, 02OV015 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, CCND1, MAP2K2 , MDM2, MTORC2, PIK3R3 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 8)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 99 samples fo

Wildtype_Tumor    46
Deletion          20
Name: Mutation, dtype: int64

In [15]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
o_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if o_sig is not None:
    o_sig_list = list(o_sig.Comparison)
else: 
    o_sig_list = None
print('significant pvals: \n',o_sig)

# Get all pvals
o = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
o_pval = o.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Ov_P_Value'})
#o_pval = o_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)
#o_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


significant pvals: 
         Comparison   P_Value
0  PTEN_proteomics  0.000909


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


In [16]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

ov_d = {}

for prot in ip:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    ov_d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

o_df = pd.DataFrame.from_dict(ov_d, orient='index', columns=['Ov_Median'])
o_df = o_df.reset_index().rename(columns={'index':'Proteomics'})
#o_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [17]:
o_merged = o_pval.merge(o_df, on='Proteomics',how='inner')
o_merged.head()

,Proteomics,Ov_P_Value,Ov_Median
0,PTEN_proteomics,0.000909,0.169972
1,PIK3R2_proteomics,0.005865,-0.240986
2,PIK3CA_proteomics,0.008335,-0.199296
3,PIK3CB_proteomics,0.050313,-0.276354
4,PREX2_proteomics,0.060296,-0.505300


Breast

In [18]:
#b = cptac.Brca()

In [19]:
mut_type = b.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = b.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ip)
prot_and_mutations = b.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations.loc[:,~prot_and_mutations.columns.duplicated()] # drop duplicated columns
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 113 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, CSNK2A1, MAP2K2 , MDM2, MTORC2, PIK3R3 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 113 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: Due to dropping the specified levels, dataframe now has 2 duplicated column headers. (C:\Users\brittany 

Wildtype_Tumor    80
Deletion          24
Name: Mutation, dtype: int64

In [20]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
b_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if b_sig is not None:
    b_sig_list = list(b_sig.Comparison)
else: 
    b_sig_list = None
print('significant pvals: \n',b_sig)

# Get all pvals
b_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
b_pval = b_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Brca_P_Value'})
#b_pval = b_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)

#b_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:147: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf


significant pvals: 
         Comparison   P_Value
0  PTEN_proteomics  0.000004


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


In [21]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

b_d = {}

for prot in ip:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    b_d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

b_df = pd.DataFrame.from_dict(b_d, orient='index', columns=['Brca_Median'])
b_df = b_df.reset_index().rename(columns={'index':'Proteomics'})
#b_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [22]:
b_merged = b_pval.merge(b_df, on='Proteomics',how='inner')
b_merged.head()

,Proteomics,Brca_P_Value,Brca_Median
0,PTEN_proteomics,0.000004,1.00800
1,SHC1_proteomics,0.014172,-0.42620
2,EGFR_proteomics,0.016614,-0.58645
3,GRB2_proteomics,0.061517,0.31470
4,TP53_proteomics,0.068427,-0.11100


Colon

In [23]:
#col = cptac.Colon()

In [24]:
mut_type = col.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = col.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ip)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 100 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, CCND1, FOXO1, MAGI3, MAP2K2 , MAST2, MDM2, MTORC2, PIK3R3, PREX2 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_N

Wildtype_Tumor    76
Deletion          25
Name: Mutation, dtype: int64

In [25]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
c_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if c_sig is not None:
    c_sig_list = list(c_sig.Comparison)
else: 
    c_sig_list = None
print('significant pvals: \n',c_sig)

# Get all pvals
c_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
c_pval = c_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Colon_P_Value'})
#c_pval = c_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)

#c_pval

significant pvals: 
 None


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


In [26]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

d = {}

for prot in ip:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

c_df = pd.DataFrame.from_dict(d, orient='index', columns=['Colon_Median'])
c_df = c_df.reset_index().rename(columns={'index':'Proteomics'})
#c_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [27]:
c_merged = c_pval.merge(c_df, on='Proteomics',how='inner')
c_merged.head()

,Proteomics,Colon_P_Value,Colon_Median
0,MAPK3_proteomics,0.003390,-0.1949
1,PTEN_proteomics,0.012872,0.2040
2,SHC1_proteomics,0.030107,-0.1161
3,PDK1_proteomics,0.050997,0.0790
4,KRAS_proteomics,0.062750,0.1056


Head and Neck

In [28]:
#h = cptac.Hnscc()

In [29]:
mut_type = h.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = h.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ip)
#prot_and_mutations = h.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 109 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-02617, C3N-02727 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, MAP2K2 , MDM2, MTORC2, PIK3R3 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 176 samples for

Wildtype_Tumor    87
Deletion          18
Name: Mutation, dtype: int64

In [30]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
h_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if h_sig is not None:
    h_sig_list = list(h_sig.Comparison)
else: 
    h_sig_list = None
print('significant pvals: \n',h_sig)

# Get all pvals

h_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
h_pval = h_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Hnscc_P_Value'})
#h_pval = h_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)

#h_pval


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

significant pvals: 
           Comparison   P_Value
0    PTEN_proteomics  0.000023
1  INPP4B_proteomics  0.000676
2  PIK3CA_proteomics  0.001677


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alp

In [31]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

h_d = {}

for prot in ip:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    h_d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

h_df = pd.DataFrame.from_dict(h_d, orient='index', columns=['Hnscc_Median'])
h_df = h_df.reset_index().rename(columns={'index':'Proteomics'})
#h_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [32]:
h_merged = h_pval.merge(h_df, on='Proteomics',how='inner')
#del_wt[['PREX2_proteomics','Mutation']].dropna() # in median, not in pval (only 1 value for deletion)
h_merged.head()

,Proteomics,Hnscc_P_Value,Hnscc_Median
0,PTEN_proteomics,0.000023,0.318812
1,INPP4B_proteomics,0.000676,0.591596
2,PIK3CA_proteomics,0.001677,-0.163663
3,USP13_proteomics,0.005645,-0.217338
4,TSC2_proteomics,0.012038,0.081507


Lung

In [33]:
#l = cptac.Luad()

In [34]:
mut_type = l.get_genotype_all_vars(gene)
mut_type = mut_type.rename(columns={'PTEN':'cnv'})

# different code because no somatic mutation data for pten (can't join to somatic mutations)
omics = l.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics',genes1="PTEN", 
    genes2=ip)
omics = l.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
omics = omics.drop(columns='PTEN_CNV')

# Get only tumor samples
p = l.get_proteomics(tissue_type='tumor')
tumor_ids = list(p.index)
get = omics.index.isin(tumor_ids)
omics = omics[get]

merged = omics.join(mut_type) # checked and there is 110 tumor samples for lung

compare = ['No_Mutation','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
print(del_wt['Mutation'].value_counts())
del_wt = del_wt.dropna(axis='columns', how='all')
del_wt = del_wt.loc[:,~del_wt.columns.duplicated()] # SH1, PRK3R1 both have double columns


cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, CSNK2A1, CSNK2A2, MAP2K2 , MDM2, MTOR, MTORC2, PIK3R3, RAF1, SOS1, TSC2 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 6)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-00001.N, C3L-00009.N, C3L-00080.N, C3L-00083.N, C3L-00093.N, C3L-00094.N, C3L-00095.N, C3L-00140.N, C3L-00144.N, C3L-00263.N, C3L-00279.N, C3L-00368.N, C3L-00412.N, C3L-00422.N, C3L-00510.N, C3L-00604.N, C3L-00893.N, C3L-00913.N, C3L-00973.N, C3L-01330.N, C3L-01632.N, C3L-01682.N, C3L-01683.N, C3L-01889.N, C3L-01890, C3L-01890.N, C3L-01924.N, C3L-02219.N, C3L-02345.N, C3L-02348.N, C3L-02350.N, C3L-02365.N, C3L-02508.N, C3L-02549.N, C3N-00167.N, C3N-00169.N, C3N-00175.N, C3N-00180.N, C3N-00199.N, C3N-00203.N, C3N-00217.N, C3N-00223.N, C3N-0029

No_Mutation    88
Deletion       17
Name: Mutation, dtype: int64


In [35]:
cols = list(del_wt.columns[:-2])

# Get only sig sites
l_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if l_sig is not None:
    l_sig_list = list(l_sig.Comparison)
else: 
    l_sig_list = None
print('significant pvals: \n',l_sig)

# Get all pvals
l_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
l_pval = l_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Luad_P_Value'})
#l_pval = l_pval.replace(to_replace = r'_NP_.*$', value = '', regex = True)
l_pval.head() # isoforms for some proteins

significant pvals: 
           Comparison       P_Value
0    EGFR_proteomics  9.121167e-07
1  PIK3CD_proteomics  4.312883e-04
2    XIAP_proteomics  4.264738e-03
3   FOXO1_proteomics  5.642364e-03


,Proteomics,Luad_P_Value
0,EGFR_proteomics,9.121167e-07
1,PIK3CD_proteomics,4.312883e-04
2,XIAP_proteomics,4.264738e-03
3,FOXO1_proteomics,5.642364e-03
4,GRB2_proteomics,7.160789e-03


In [36]:
#Isoform code - fix later
#del_wt.replace(to_replace = r'_NP_.*$', value = '', regex = True)

In [37]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "No_Mutation"]
del_med = d.median()
wt_med = wt.median()

l_d = {}

for prot in cols:
    dif = wt_med[prot] - del_med[prot]
    l_d[prot] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

l_df = pd.DataFrame.from_dict(l_d, orient='index', columns=['Luad_Median'])
l_df = l_df.reset_index().rename(columns={'index':'Proteomics'})
l_df.head()

,Proteomics,Luad_Median
0,AKT1_proteomics,-0.28010
1,BAD_proteomics,0.05135
2,CCND1_proteomics,1.18450
3,EGFR_proteomics,-2.32970
4,FOXO1_proteomics,0.57005


In [38]:
l_merged = l_pval.merge(l_df, on='Proteomics',how='inner')
#l_merged = l_merged.replace(to_replace = r'_proteomics$', value = '', regex = True)
l_merged.head()

,Proteomics,Luad_P_Value,Luad_Median
0,EGFR_proteomics,9.121167e-07,-2.32970
1,PIK3CD_proteomics,4.312883e-04,0.84795
2,XIAP_proteomics,4.264738e-03,-0.52480
3,FOXO1_proteomics,5.642364e-03,0.57005
4,GRB2_proteomics,7.160789e-03,0.55885


In [39]:
n = g_merged.merge(h_merged, on='Proteomics',how='outer')
n = n.merge(l_merged, on='Proteomics',how='outer')
n= n.merge(b_merged, on='Proteomics',how='outer')
n = n.merge(o_merged, on='Proteomics',how='outer')
n = n.merge(e_merged, on='Proteomics',how='outer')
all_df = n.merge(c_merged, on='Proteomics',how='outer')


In [40]:
# Keep only genes significant in > 1 cancer
all_sig = []

sig_lists = [g_sig_list, e_sig_list, b_sig_list, o_sig_list, c_sig_list, h_sig_list, l_sig_list]

for next_list in sig_lists:
    if next_list is not None:
        all_sig.append(next_list)
        
flat_list = [item for sublist in all_sig for item in sublist]
sig = set(flat_list) # remove duplicates


bool_df = all_df['Proteomics'].isin(sig)
sig_df = all_df[bool_df]
print(len(sig_df))
sig_df

15


,Proteomics,Gbm_P_Value,Gbm_Median,Hnscc_P_Value,Hnscc_Median,Luad_P_Value,Luad_Median,Brca_P_Value,Brca_Median,Ov_P_Value,Ov_Median,En_P_Value,En_Median,Colon_P_Value,Colon_Median
0,PTEN_proteomics,1.219523e-07,0.511726,0.000023,0.318812,1.435202e-02,0.36490,0.000004,1.00800,0.000909,0.169972,0.000640,0.74660,0.012872,0.20400
1,MAPK3_proteomics,1.020660e-06,0.665105,0.129494,0.061417,5.256187e-01,-0.33845,0.285412,0.22335,0.842428,-0.059507,0.272111,-0.08200,0.003390,-0.19490
2,PIK3CD_proteomics,1.030815e-03,0.492099,0.014016,0.238171,4.312883e-04,0.84795,0.836701,-0.15015,0.937984,0.035308,0.045489,0.18243,0.131200,0.07600
3,GRB2_proteomics,1.089951e-03,0.312146,0.104723,0.190047,7.160789e-03,0.55885,0.061517,0.31470,0.086428,-0.184243,0.160116,0.26400,0.827435,-0.01680
4,PIK3C3_proteomics,1.225949e-03,0.102518,0.312904,0.041287,5.491624e-01,0.10100,0.445080,-0.11490,0.444583,-0.104687,0.036592,0.13160,0.116419,0.07995
5,USP7_proteomics,4.017792e-03,-0.154035,0.064384,-0.094081,3.715141e-01,-0.13260,0.929331,0.14305,0.525069,0.017808,0.572500,-0.03834,0.237441,-0.04450
6,MAP2K1_proteomics,4.138456e-03,0.592389,0.550116,-0.008519,2.426636e-01,0.10475,0.757927,0.03045,0.472124,0.025625,0.675793,0.09190,0.717025,0.02675
7,PIK3R1_proteomics,4.282296e-03,0.188390,0.434622,-0.072064,6.486765e-01,-0.47740,0.164475,1.03615,0.219977,-0.179425,0.135943,-0.33990,0.972777,0.01245
9,INPP4B_proteomics,1.455578e-02,0.341829,0.000676,0.591596,2.766716e-01,-1.87510,0.295970,0.37695,0.353620,-0.370647,0.489186,-0.16320,0.118135,-0.08220
12,EGFR_proteomics,4.010871e-02,-1.001035,0.017719,-0.467227,9.121167e-07,-2.32970,0.016614,-0.58645,0.838960,0.063388,0.006128,-0.52600,0.366886,0.02350


In [41]:
sig_df.to_csv('sig_pval_15_interacting.csv')

In [42]:
# Lists sig in cancer
cancer = ['Gbm','En','Brca','Ov','Colon','Hnscc','Luad']
i = 0
for next_list in sig_lists:

    print(cancer[i])
    print(next_list, '\n')
    if (i < 6):
        i += 1
    

Gbm
['PTEN_proteomics', 'MAPK3_proteomics', 'PIK3CD_proteomics', 'GRB2_proteomics', 'PIK3C3_proteomics', 'USP7_proteomics', 'MAP2K1_proteomics', 'PIK3R1_proteomics'] 

En
['PTEN_proteomics', 'TP53_proteomics', 'PIK3CA_proteomics', 'CSNK2A1_proteomics'] 

Brca
['PTEN_proteomics'] 

Ov
['PTEN_proteomics'] 

Colon
None 

Hnscc
['PTEN_proteomics', 'INPP4B_proteomics', 'PIK3CA_proteomics'] 

Luad
['EGFR_proteomics', 'PIK3CD_proteomics', 'XIAP_proteomics', 'FOXO1_proteomics'] 



# HeatMaps


In [55]:
# Create df for heat map

cancer = ['Gbm','Hnscc','Luad','Brca','Ov','En','Colon']
merged_dfs = [g_merged,h_merged,l_merged,b_merged,o_merged,e_merged,c_merged]

merged = pd.DataFrame()
i = 0
for c in cancer:
    m = merged_dfs[i]
    m2 = m.assign(Cancer = c)
    m2 = m2.rename(columns={c+'_P_Value': 'P_Value'})
    m2 = m2.rename(columns={c+'_Median': 'Medians'})
    merged = merged.append(m2)
    if i < 6:
        i += 1
        
bool_df2 = merged['Proteomics'].isin(sig)
plot_df = merged[bool_df2]
plot_df = plot_df.replace(to_replace = r'_proteomics$', value = '', regex = True)

# log p-vals for right scale in plot (bigger circle, smaller pval)
plot_df['size'] = plot_df['P_Value'].apply(lambda x: -1*(np.log(x)/10))

plot_df

,Proteomics,P_Value,Medians,Cancer,size
0,PTEN,1.219523e-07,0.511726,Gbm,1.591964
1,MAPK3,1.020660e-06,0.665105,Gbm,1.379506
2,PIK3CD,1.030815e-03,0.492099,Gbm,0.687741
3,GRB2,1.089951e-03,0.312146,Gbm,0.682162
4,PIK3C3,1.225949e-03,0.102518,Gbm,0.670404
...,...,...,...,...,...
29,MAP2K1,7.170253e-01,0.026750,Colon,0.033264
31,GRB2,8.274353e-01,-0.016800,Colon,0.018942
33,TP53,9.688700e-01,-0.049050,Colon,0.003162
34,PIK3R1,9.727770e-01,0.012450,Colon,0.002760


In [56]:
pu.plotCircleHeatMap(plot_df, circle_var = 'size', color_var='Medians', x_axis= 'Cancer', y_axis = 'Proteomics')

Loading BokehJS ...

Keep PosNeg Genes

In [57]:
def HasPosNeg(row):
    hasPos = False
    hasNeg= False

    for item in row:
        if pd.isnull(item):
            continue
        if item < -0.3:
            hasNeg = True
        if item > 0.3:
            hasPos = True
           
    if hasPos & hasNeg:
        return True
    return False


In [59]:
sig_df = sig_df.replace(to_replace = r'_proteomics$', value = '', regex = True)


pt = sig_df.set_index('Proteomics')
pt = pt.drop(columns= ['Gbm_P_Value','Hnscc_P_Value','Luad_P_Value',
                     'Brca_P_Value','Ov_P_Value','En_P_Value','Colon_P_Value'])
pt.head()

,Gbm_Median,Hnscc_Median,Luad_Median,Brca_Median,Ov_Median,En_Median,Colon_Median
Proteomics,,,,,,,
PTEN,0.511726,0.318812,0.36490,1.00800,0.169972,0.74660,0.20400
MAPK3,0.665105,0.061417,-0.33845,0.22335,-0.059507,-0.08200,-0.19490
PIK3CD,0.492099,0.238171,0.84795,-0.15015,0.035308,0.18243,0.07600
GRB2,0.312146,0.190047,0.55885,0.31470,-0.184243,0.26400,-0.01680
PIK3C3,0.102518,0.041287,0.10100,-0.11490,-0.104687,0.13160,0.07995


In [61]:
pt["Pos_Neg"] = pt.apply(HasPosNeg, axis = 1)

pn = pt.loc[pt['Pos_Neg'] == True]
pn_genes = list(pn.index)

get = plot_df['Proteomics'].isin(pn_genes)
plot_df2 = plot_df[get]

plot_df2['size'] = plot_df2['P_Value'].apply(lambda x: -1*(np.log(x)/10))
plot_df2.head()

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Proteomics,P_Value,Medians,Cancer,size
1,MAPK3,0.000001,0.665105,Gbm,1.379506
7,PIK3R1,0.004282,0.188390,Gbm,0.545327
9,INPP4B,0.014556,0.341829,Gbm,0.422977
35,TP53,0.668250,-0.256630,Gbm,0.040309
1,INPP4B,0.000676,0.591596,Hnscc,0.729998


In [65]:
pu.plotCircleHeatMap(plot_df2, circle_var = 'size', color_var='Medians', x_axis= 'Cancer', y_axis = 'Proteomics')

C:\Users\brittany henderson\GitHub\WhenMutationsDontMatter\plot_utils.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = np.where(df[circle_var]<0, np.abs(df[circle_var]), df[circle_var])*50


Loading BokehJS ...

Extras

In [ ]:
plotCircleHeatMap(plot_df, circle_var = 'Medians', color_var='Medians', x_axis= 'Cancer', y_axis = 'Proteomics')

In [ ]:
# Code for only keeping data for sig pvals
# 0 for not sig (no circle)
bc = .05/46
c = .05/36
r = .01

plot_df['size'] = plot_df['P_Value'].apply(lambda x: .75 if x <= bc else (.5 if x <= c else (.25 if x <= r else 0)))

plot_df

In [ ]:
p.plotCircleHeatMap(plot_df, circle_var = 'size', color_var='Medians', x_axis= 'Cancer', y_axis = 'Proteomics')

Combine

In [ ]:
sig_df.to_csv('at_least_one_significant_interacting.csv')

In [ ]:
stdev = ip_df.std()

In [ ]:
stdev = stdev.to_frame()

In [ ]:
stdev.loc[stdev[0] > .2]

In [ ]:
d_stdev = stdev.std()
d_stdev

In [ ]:
a = stdev.hist(bins = 20)
plt.title("Standard Deviations of Pancancer CNV deletions and amplifications of PTEN on Interacting Proteins\n")
plt.xlabel('StDeV')
plt.ylabel('Count')

In [ ]:
plt.rcParams['figure.figsize']=(10,8)
sns.set(font_scale = 1.3)
a = sns.distplot(stdev, bins = 20)

a.set_title("Standard Deviation Distribution for Pancancer PTEN Mutation Effect on Proteomics\n\n (Median of proteomics with Wildtype PTEN - Median of proteomics with Mutated PTEN)")
a.set(xlabel = 'StDeV', ylabel = 'Frequency')

Mean

In [ ]:
mean_df = stdev.mean()

In [ ]:
a = mean_df.hist(bins = 20)
plt.title("Distribution of the Mean between Cancers for PTEN Mutation Effect on all Proteins\n\n (Median of proteomics with Wildtype PTEN - Median of proteomics with Mutated PTEN)")
plt.xlabel('Mean')
plt.ylabel('Count')

In [ ]:
m_df = stdev.median()

In [ ]:
a = m_df.hist(bins = 20)
plt.title("Distribution of the Medians between Cancers for PTEN Mutation Effect on all Proteins\n\n (Median of proteomics with Wildtype PTEN - Median of proteomics with Mutated PTEN)")
plt.xlabel('Median')
plt.ylabel('Count')